機械学習特論 第6回 決定木 で mnist を分類

In [ ]:
# google colab で実行する場合は、次の行の先頭の # を削除してこのブロックを実行する
#!pip install japanize-matplotlib

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn import metrics

In [ ]:
# sklearnデータセットに収録されたiris(アヤメ)のデータセットをロード
from sklearn.datasets import fetch_openml
# 手書き文字のデータセットをダウンロードして、実験用データを準備 (70000枚のうち7000枚を利用)
mnist_data = fetch_openml('mnist_784')
_x = np.array(mnist_data['data'].astype(np.float32))
_y = np.array(mnist_data['target'].astype(np.int32))
_, x, _, y = train_test_split(_x, _y, test_size=0.1, random_state=1)

In [ ]:
# データを学習用と検証用に分割
x_train, x_test, y_train, y_test = \
    train_test_split(x, y, test_size=0.25, random_state=2) # 検証用データに25%を割当て
print(f'x_train:{len(x_train)} x_test:{len(x_test)} y_train:{len(y_train)} y_test:{len(y_test)}')

In [ ]:
# 決定木を学習データを利用して学習
clf = DecisionTreeClassifier(max_depth=1, # 木の深さの最大
                             random_state=2) # 乱数シード
clf = clf.fit(x_train, y_train)

# 学習したモデルの性能(正答率)を学習用データと検証用データで評価
predict_train = clf.predict(x_train)
predict_test = clf.predict(x_test)
print('max_depth=1, accuracy_score: ', 
      f'train data : {metrics.accuracy_score(y_train, predict_train): 0.5}', 
      f'test data : {metrics.accuracy_score(y_test, predict_test): 0.5}')

# 決定木を表示
plt.figure(figsize=[15,8])
plot_tree(clf, filled=True)
plt.show()

In [ ]:
# 木の深さを変えて学習した決定木の性能を学習用データと検証用データで確認
N = 20
_x = np.linspace(1, N, N) # グラフのx軸の設定 (1から7までの7点)
train_score = [] # グラフ用のリスト
test_score = [] # グラフ用のリスト
for i in range(N): # 木の深さは 1-7 (iに1を足して利用)
    # 決定木を学習データを利用して学習
    clf = DecisionTreeClassifier(max_depth=i+1, # 木の深さの最大
                                random_state=1) # 乱数シード
    clf = clf.fit(x_train, y_train)  # * y_trainの列指定はwarning回避のため *

    # 学習したモデルの性能(正答率)を学習用データと検証用データで評価
    # 計算した性能はグラフ用のリストに格納
    predict_train = clf.predict(x_train)
    train_score.append(metrics.accuracy_score(y_train, predict_train))
    predict_test = clf.predict(x_test)
    test_score.append(metrics.accuracy_score(y_test, predict_test))
    print(f'max_depth={i+1}, accuracy_score: ', 
          f'train data : {train_score[i]: 0.5}', 
          f'test data : {test_score[i]: 0.5}')

# 木の深さに対する決定木の性能をグラフで表示
plt.figure(figsize=[10,6])
plt.plot(_x, train_score, label='train_score')
plt.plot(_x, test_score, label='test_score')
plt.legend()
plt.xticks(_x)
plt.show()

In [ ]:
# 決定木を学習データを利用して学習
clf = DecisionTreeClassifier(max_depth=10, # 木の深さの最大
                             random_state=2) # 乱数シード
clf = clf.fit(x_train, y_train)

# 学習したモデルの性能(正答率)を学習用データと検証用データで評価
predict_train = clf.predict(x_train)
predict_test = clf.predict(x_test)
print('max_depth=10, accuracy_score: ', 
      f'train data : {metrics.accuracy_score(y_train, predict_train): 0.5}', 
      f'test data : {metrics.accuracy_score(y_test, predict_test): 0.5}')

# 画素の重要度を確認
importance = {}
img = []
zero_importance = 0
for i in range(len(clf.feature_importances_)):
    importance[i] = clf.feature_importances_[i]
    img.append(clf.feature_importances_[i])
    if clf.feature_importances_[i] == 0.0:
        zero_importance = zero_importance + 1
print('重要度がゼロの画素数:', zero_importance, '/', len(importance), f'({zero_importance/len(importance)})')
print('重要度が高い画素(上位20件):')
importance_sorted = sorted(importance.items(), key=lambda x:x[1], reverse=True)
i = 0
cumulative_importance = 0
for _importance_sorted in importance_sorted:
    cumulative_importance = cumulative_importance + _importance_sorted[1]
    print(i, _importance_sorted, cumulative_importance)
    i = i + 1
    if i > 100:
        break

plt.imshow(np.array(np.log(img)).reshape(28,28), cmap=plt.cm.gray_r)
plt.xticks(np.linspace(0,27,28))
plt.yticks(np.linspace(0,27,28))
plt.xticks(color="None")
plt.yticks(color="None")
plt.grid(linewidth=0.5)
plt.show()

plt.figure(figsize=[10,4])
plt.bar(np.linspace(0,len(img),len(img)), sorted(img, reverse=True), width=1.0)
plt.ylim(0,0.06)
plt.show()